<a href="https://colab.research.google.com/github/sinajahangir/FloodAgent/blob/main/FloodAgentDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This AI agent code is designed to extract location parameters (coordinates) from a user prompt using natural language processing. After identifying the relevant location, the agent queries a flood information system to retrieve and present current flood-related details (e.g., severity, vulnerability) specific to the extracted location.
The case study presented in this notebook is associated with City of Calgary

You do not need to run the notebook on GPU

Developed by: Sina Jahangir (first version: October 2025)
license: GNU GENERAL PUBLIC LICENSE

# Clone repository

After executing, you should see the repo in Files section

In [22]:
import os
if os.path.isdir('FloodAgent'):
    !rm -rf FloodAgent #removed if already there is a cloned repo
!git clone https://github.com/sinajahangir/FloodAgent.git

Cloning into 'FloodAgent'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 36 (delta 15), reused 25 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 3.54 MiB | 3.86 MiB/s, done.
Resolving deltas: 100% (15/15), done.


# Install libraries and load dependencies

In [21]:
# Execute the setup.py file that is now in the cloned directory
%run FloodAgent/SetupLibraries.py

Success: All core packages were already installed and imported.
Numpy version: 2.0.2
Setup script finished execution


# LLM setup

In [9]:
# --- LLM Setup ---
try:
    # Using Colab Secrets for API key management
    api_key = userdata.get('RAG') #API key needs setup in the cloud. "RAG" is an arbitrary name example
    genai.configure(api_key=api_key)
    print("Gemini API Key configured.")
    # Initialize the generative model
    llm_model = genai.GenerativeModel('gemini-2.5-flash-lite') # Use a model good at instruction following/extraction
    print(f"LLM Model '{llm_model.model_name}' initialized.")
except userdata.SecretNotFoundError:
    print("ERROR: Gemini API Key ('GEMINI_API_KEY') not found in Colab Secrets.")
    print("Please add your API key via the 'Secrets' tab (key icon) on the left.")
    llm_model = None
except Exception as e:
    print(f"An error occurred during Gemini setup: {e}")
    llm_model = None

Gemini API Key configured.
LLM Model 'models/gemini-2.5-flash-lite' initialized.


# Agent class

In [17]:
from FloodAgent.AgentSource import CoordinateFloodProximityAgent

# Read data

In [23]:
df_flood=pd.read_csv('FloodAgent/Data/CA_202012_FLSW_U_RP100_RB_30m_4326.csv') #flood data
df_sovi=pd.read_csv('FloodAgent/Data/DA_SoVI_Coordinates.csv') #Social Vulnerability
modeli_=DiscreteBayesianNetwork.load('FloodAgent/Data/BBN_Model_RoadNetwork_Updated.bif', filetype='bif') #Bayesian network

In [24]:
#Convert dataframes to numpy array (expected by the agent)
flood_pixels = np.array(df_flood[['easting', 'northing']])
sovi_pixels = np.array(df_sovi[['easting', 'northing']])
sovi_array = np.array(df_sovi['SoVI'])
pop_density_array = np.array(df_sovi['pop_d_c'])

#Test Agent

In [27]:
# --- Run Example IF LLM is available and data is ready ---
if llm_model and len(flood_pixels) > 0:
    # Instantiate the agent
    try:
        agent = CoordinateFloodProximityAgent(
            flood_pixel_coords=flood_pixels,# Essential: Flood pixel data
            sovi_coords=sovi_pixels,# SoVI coordination
            sovi_array=sovi_array, # SoVI data
            pop_density_array=pop_density_array,
            BBN=modeli_, #Bayesian network,
            llm_model=llm_model #LLM model
        )

        prompt1 = "Explain the flooding situation at largest hospital in Calgary, Foothills Medical Centre, located at coordinates 51.06N, 114.13W?"
        print(f"\nUser Query 1: {prompt1}")
        response1 = agent.find_closest_flood_pixel_to_location(prompt1)
        print(f"Agent Response 1:\n{response1}")

    except Exception as e:
        print(f"\nAn error occurred during agent instantiation or testing: {e}")

elif not llm_model:
     print("\nSkipping example usage: LLM model not initialized.")
else:
     print("\nSkipping example usage: Placeholder data (buildings or flood pixels) is empty or missing.")

Stored 191478 flood pixel coordinates.
Building k-d tree for flood pixels...
Flood k-d tree built.

--- Testing Queries ---

User Query 1: Explain the flooding situation at largest hospital in Calgary, Foothills Medical Centre, located at coordinates 51.06N, 114.13W?
Converted to UTM (EPSG:32611): Easting=701109.2957626152, Northing=5660416.328740194
Closest flood pixel found: Coords=[ 700989.96228624 5660211.09291781], Distance=237.4073
Agent Response 1:
Here's a concise response to the user, incorporating the provided information and estimating flood resilience:

**Your location is near a recorded flood event.**

The closest flood data point is approximately **237 meters away**. This area has a **Low** social vulnerability (SoVI value class) and a **Moderate** population density, with an expected local road class. Considering these factors, the overall flood exposure is assessed as **Moderate**.

**Flood Resilience Estimate:**

Given the **Low social vulnerability**, **Moderate popul